# Como realizar el analisis de datos en Jupyter 

[PyCon Ar en PyVideo.org](https://pyvideo.org/events/pycon-ar-2011.html)

- Explorar los datos

In [19]:
import pathlib

carpeta_datos = pathlib.Path('./datos')

In [33]:
archivos = [ elemento for elemento in carpeta_datos.glob('*/videos/*.json') ]

In [34]:
len(archivos)

237

In [35]:
{archivo.suffix for archivo in archivos }

{'.json'}

In [31]:
archivos[0].suffix

''

- Importar los datos con pandas

In [36]:
import pandas

In [58]:
def safe_df(archivo):
    try:
        df = pandas.read_json(archivo,typ='series')
    except ValueError as err:
        df = pandas.DataFrame()
        print('Fallo al cargar: ', archivo)
    return df

In [59]:
series = [ safe_df(archivo) for archivo in archivos ]

In [69]:
df = pandas.concat(series,axis=1).T

In [73]:
speakers = df.speakers.explode()

In [82]:
import bqplot.pyplot as plt

pandas.set_option('plotting.backend', 'bqplot.pyplot')

In [83]:
df.speakers.apply(len).sort_values().plot()
plt.show()

In [127]:
import bqplot

In [164]:
x=df.speakers.explode().value_counts()
x =  x[x.between(2,10)]
plt.figure()
plt.scales(scales={'x':bqplot.OrdinalScale(),'y':plt.LinearScale()})
bar = plt.bar(x.index,x.values,orientation='horizontal')
plt.show()

In [147]:
df.recorded = pandas.to_datetime( df.recorded  )

In [161]:
recorded = df.recorded.dt.year.value_counts()
plt.figure()
plt.scatter('index','recorded',data=recorded.reset_index())
plt.xlabel('Anio')
plt.ylabel('Cantidad de charlas')
plt.show()

In [170]:
df.description.str.split(' ')

0      [Alan, Runyan, y, su, charla, en, el, auditori...
1      ["Los, Diferidos", es, posiblemente, el, tema,...
2      [Descripción, de, la, charla:, DBus, permite, ...
3      [Descripción, de, la, charla:, Cocción, de, un...
4      [Arduino, es, una, plataforma, de, hardware, y...
                             ...                        
232    [Usamos, Python, para, la, conversión, de, la,...
233    [Keynote, presented, on, Saturday, in, the, Py...
234    [Reescribir, una, gran, parte, de, un, proyect...
235    [Machine, learning, productivo,, en, el, mundo...
236    [Async, está, de, moda,, y, todo, el, mundo, d...
Name: description, Length: 237, dtype: object

In [184]:
df[ df.title.str.contains('python') ]

,description,language,recorded,speakers,thumbnail_url,title,videos,tags,copyright_text,duration,related_urls,alias,category,id,quality_notes,slug,summary
64,Over the last decade Brazilian government embr...,spa,2012-11-17,[Erico Andrei],https://i.ytimg.com/vi/qqleysomo14/maxresdefau...,How Brazil is building a digital nation with o...,"[{'type': 'youtube', 'url': 'https://www.youtu...",[],Creative Commons Attribution license (reuse al...,2477,"[{'label': 'Event schedule', 'url': 'https://w...",NaN,NaN,NaN,NaN,NaN,NaN
70,"Se introducirá aima-python, una biblioteca Pyt...",spa,2012-11-16,[Rafael Carrascosa],https://i.ytimg.com/vi/da2l8tYWfgs/maxresdefau...,"Inteligencia Artificial con aima-python, un re...","[{'type': 'youtube', 'url': 'https://www.youtu...",[],Creative Commons Attribution license (reuse al...,2464,"[{'label': 'Event schedule', 'url': 'https://w...",NaN,NaN,NaN,NaN,NaN,NaN
75,Muchos de nosotros programamos Python pero nos...,spa,2012-11-17,[Daniel F. Moisset],https://i.ytimg.com/vi/x7QMAdFnESA/maxresdefau...,Javascript para programadores python,"[{'type': 'youtube', 'url': 'https://www.youtu...",[],Creative Commons Attribution license (reuse al...,2673,"[{'label': 'Event schedule', 'url': 'https://w...",NaN,NaN,NaN,NaN,NaN,NaN
81,Tomar ventaja del multiprocesamiento de los si...,spa,2012-11-17,[Claudio Freire],https://i.ytimg.com/vi/ZH4z4KbbpVk/maxresdefau...,Multiprocesamiento en python,"[{'type': 'youtube', 'url': 'https://www.youtu...",[],Creative Commons Attribution license (reuse al...,2668,"[{'label': 'Event schedule', 'url': 'https://w...",NaN,NaN,NaN,NaN,NaN,NaN
82,En el mundo del hardware se usan lenguajes de ...,spa,2012-11-17,[Martín Gaitán],https://i.ytimg.com/vi/05HfBi2skiw/maxresdefau...,"MyHDL, de python al silicio","[{'type': 'youtube', 'url': 'https://www.youtu...",[],Creative Commons Attribution license (reuse al...,2688,"[{'label': 'Event schedule', 'url': 'https://w...",NaN,NaN,NaN,NaN,NaN,NaN
83,En esta charla vamos a aprender cómo desarroll...,spa,2012-11-17,[Ezequiel Gutesman],https://i.ytimg.com/vi/hN-STUjbzY8/maxresdefau...,No toca BOTOn: Amazon Web Services desde python,"[{'type': 'youtube', 'url': 'https://www.youtu...",[],Creative Commons Attribution license (reuse al...,3327,"[{'label': 'Event schedule', 'url': 'https://w...",NaN,NaN,NaN,NaN,NaN,NaN
97,| Usando python y unittest/testtools como fram...,spa,2012-11-17,[Sergio Schvezov],https://i.ytimg.com/vi/BWF9N18Qy2Q/maxresdefau...,Testing funcional con python,"[{'type': 'youtube', 'url': 'https://www.youtu...",[],Creative Commons Attribution license (reuse al...,1570,"[{'label': 'Event schedule', 'url': 'https://w...",NaN,NaN,NaN,NaN,NaN,NaN
104,Durante la charla se presentará una introducci...,spa,2013-10-24,[Ariel Rossanigo],https://i.ytimg.com/vi/KGoNhV_l940/hqdefault.jpg,"Aprendizaje por refuerzo, una versión pythonica","[{'type': 'youtube', 'url': 'https://www.youtu...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,"¿Qué significa que un código sea ""pythónico""? ...",spa,2013-10-24,[Martín Gaitán],https://i.ytimg.com/vi/dByxE2C-daI/hqdefault.jpg,La pythonicidad al palo,"[{'type': 'youtube', 'url': 'https://www.youtu...",NaN,NaN,NaN,"[{'label': 'talk slides', 'url': 'https://gith...",NaN,NaN,NaN,NaN,NaN,NaN
164,PyconAR 2016 - Bahía Blanca\n\nMicropython en ...,spa,2016-11-25,[Martin Ribelotta],https://i.ytimg.com/vi/pT-AOUzZBsw/maxresdefau...,Micropython en EDU-CIAA,"[{'type': 'youtube', 'url': 'https://www.youtu...",[],Standard YouTube License,2375,[http://ar.pycon.org/proposals/document/24/02f...,NaN,NaN,NaN,NaN,NaN,NaN


In [188]:
df.title.str.lower().str.replace(':','3')

0                                                  plone
1                                   todo sobre deferreds
2                                dbus con el pie derecho
3                  plone 4: tiempo de cocción 30 minutos
4                          robótica con python y arduino
                             ...                        
232    reverse-engineering 170 mil líneas de document...
233                   keynote: sharing, by carol willing
234    sobreviviendo al síndrome del segundo sistema ...
235             testing en proyectos de machine learning
236                 todo lo que no querés saber de async
Name: title, Length: 237, dtype: object

In [189]:
import string

In [190]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [192]:
string.ascii_lowercase

'abcdefghijklmnopqrstuvwxyz'

In [249]:
titulos = ( df.title
    .str.lower()
    .str.replace('\W',' ',regex=True)
    .str.replace('\s+',' ',regex=True)
    .str.translate(str.maketrans('áéíóúñ','aeioun'))
)

In [224]:
df.iloc[:3].title.str.translate(str.maketrans('aeiou','AEIOU'))

0                      PlOnE
1       TOdO sObrE dEfErrEds
2    DBUs cOn El pIE dErEchO
Name: title, dtype: object

In [243]:
set(''.join( x for x in df.title.str.lower() ) )

{' ',
 '!',
 '"',
 '%',
 "'",
 '(',
 ')',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '=',
 '?',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '¡',
 '¿',
 'á',
 'é',
 'í',
 'ñ',
 'ó',
 'ú',
 '‘',
 '’',
 '“',
 '”'}

In [322]:
(titulos.str.split(' ',expand=True)
        .stack()
#         .reset_index(drop=True)
        .value_counts()
        .to_frame(name='longitud')
        .reset_index()
)
#         .groupby('palabras',as_index=False).count())

,index,longitud
0,de,56
1,python,52
2,,48
3,con,46
4,11,30
...,...,...
711,paralela,1
712,intento,1
713,artificial,1
714,durar,1


In [317]:
a = _

In [292]:
a['longitud'] = a['palabras'].apply(len)

In [297]:
set(a.loc[a.longitud<3,'palabras'])

{'',
 '0',
 '00',
 '01',
 '02',
 '03',
 '05',
 '06',
 '07',
 '08',
 '1',
 '10',
 '11',
 '12',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '24',
 '25',
 '28',
 '29',
 '3',
 '30',
 '32',
 '35',
 '36',
 '3d',
 '4',
 '40',
 '42',
 '44',
 '45',
 '46',
 '48',
 '50',
 '52',
 '54',
 '55',
 '56',
 '57',
 '59',
 '7',
 '90',
 'a',
 'al',
 'at',
 'be',
 'br',
 'by',
 'c',
 'cv',
 'd',
 'de',
 'e',
 'el',
 'en',
 'es',
 'fu',
 'is',
 'it',
 'la',
 'lo',
 'me',
 'mi',
 'no',
 'o',
 'of',
 's',
 'so',
 't',
 'to',
 'tu',
 'ui',
 'un',
 'up',
 'va',
 'wx',
 'y'}

In [314]:
a

,palabras,longitud
0,,48
1,0,1
2,00,4
3,00001_720i_fast,1
4,00002_720i_fast,1
...,...,...
711,xxi,1
712,y,26
713,yatel,1
714,your,1


In [302]:
a

,palabras,longitud
0,plone,5
1,todo,4
2,sobre,5
3,deferreds,9
4,dbus,4
...,...,...
1555,no,2
1556,queres,6
1557,saber,5
1558,de,2


In [318]:
a.groupby('palabras',as_index=False).count()

IndexError: list index out of range